In [41]:
# Import libraries
import pandas as pd
import folium

In [42]:
# Read in CSV
attendance_df = pd.read_csv("resources/attendance_and_locations.csv")

In [43]:
# Display the data
attendance_df.head()

,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Latitude,Longitude
0,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479,NaN,NaN
1,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884,NaN,NaN
2,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277,NaN,NaN
3,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314,NaN,NaN
4,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518,NaN,NaN


In [44]:
# Remove state of Connecticut from District column
districts_df = attendance_df[(attendance_df['District name'] != 'Connecticut')]
districts_df.head(40)

,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Latitude,Longitude
13,Andover School District,NaN,All Students,161,0.9386,144.0,0.9680,158.0,0.9502,41.728585,-72.366582
14,Andover School District,Students With Disabilities,Students With Disabilities,23,0.9315,NaN,NaN,NaN,NaN,41.728585,-72.366582
15,Andover School District,Free/Reduced Lunch,Free/Reduced Price Meal Eligible,30,0.9274,31.0,0.9414,37.0,0.9401,41.728585,-72.366582
16,Andover School District,Race/Ethnicity,White,134,0.9365,115.0,0.9699,128.0,0.9479,41.728585,-72.366582
17,Andover School District,High Needs,Students Without High Needs,113,0.9410,100.0,0.9752,108.0,0.9552,41.728585,-72.366582
18,Andover School District,High Needs,Students With High Needs,48,0.9328,44.0,0.9513,50.0,0.9389,41.728585,-72.366582
19,Ansonia School District,NaN,All Students,2139,0.9045,2153.0,0.8923,2185.0,0.9413,41.341123,-73.084295
20,Ansonia School District,Students With Disabilities,Students With Disabilities,369,0.8897,394.0,0.8692,411.0,0.9337,41.341123,-73.084295
21,Ansonia School District,Free/Reduced Lunch,Free Meal Eligible,1192,0.8893,1217.0,0.8657,1312.0,0.9312,41.341123,-73.084295
22,Ansonia School District,Free/Reduced Lunch,Reduced Price Meal Eligible,225,0.9282,213.0,0.9234,147.0,0.9563,41.341123,-73.084295


In [45]:
# Normalize the student count for dot size scaling
min_radius = 5  # Minimum size for a dot
max_radius = 15  # Maximum size for a dot
student_counts = districts_df['2021-2022 student count - year to date']
scaled_sizes = ((student_counts - student_counts.min()) / (student_counts.max() - student_counts.min())) * (max_radius - min_radius) + min_radius

# Reset indices to ensure they match
districts_df = districts_df.reset_index(drop=True)
scaled_sizes = scaled_sizes.reset_index(drop=True)

In [46]:
# Calculate the center of the map
center_lat = districts_df['Latitude'].mean()
center_lon = districts_df['Longitude'].mean()

# Initialize the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

In [47]:
# Define color based on attendance rate with custom ranges
def get_color(attendance_rate):
    if attendance_rate < 90:
        return 'red'  # Below 90%
    elif attendance_rate <= 91:
        return 'orange'  # Up to 91%
    elif attendance_rate <= 92:
        return 'yellow'  # Up to 92%
    elif attendance_rate <= 93:
        return 'lightgreen'  # Up to 93%
    elif attendance_rate <= 94:
        return 'green'  # Up to 94%
    else:
        return 'darkgreen'  # More than 94%

In [48]:
# Create FeatureGroups for each student group
all_students_group = folium.FeatureGroup(name='All Students')
high_needs_group = folium.FeatureGroup(name='Students With High Needs')
without_high_needs_group = folium.FeatureGroup(name='Students Without High Needs')

In [49]:
# Add markers for "All Students"
for index, row in districts_df[districts_df['Student group'] == 'All Students'].iterrows():
    district_name = row['District name']
    attendance_rate = row['2021-2022 attendance rate - year to date'] * 100  # Convert to percentage
    latitude = row['Latitude']
    longitude = row['Longitude']
    student_count = row['2021-2022 student count - year to date']
    
    # Ensure we have a matching size for the dot
    dot_size = scaled_sizes.iloc[index] if index in scaled_sizes.index else 5  # Default size if mismatch
    
    # Get the color based on attendance rate
    dot_color = get_color(attendance_rate)
    
    tooltip_text = f"District: {district_name}<br>Attendance Rate: {attendance_rate}%"

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=dot_size,  # Adjust size of the dot
        color=dot_color,  # Border color
        fill=True,
        fill_color=dot_color,  # Fill color
        fill_opacity=0.7,
        tooltip=tooltip_text
    ).add_to(all_students_group)

In [50]:
# Add markers for "Students With High Needs"
for index, row in districts_df[districts_df['Student group'] == 'Students With High Needs'].iterrows():
    district_name = row['District name']
    attendance_rate = row['2021-2022 attendance rate - year to date'] * 100  # Convert to percentage
    latitude = row['Latitude']
    longitude = row['Longitude']
    student_count = row['2021-2022 student count - year to date']
    
    # Ensure we have a matching size for the dot
    dot_size = scaled_sizes.iloc[index] if index in scaled_sizes.index else 5  # Default size if mismatch
    
    # Get the color based on attendance rate
    dot_color = get_color(attendance_rate)
    
    tooltip_text = f"District: {district_name}<br>Attendance Rate: {attendance_rate}%"

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=dot_size,  # Adjust size of the dot
        color=dot_color,  # Border color
        fill=True,
        fill_color=dot_color,  # Fill color
        fill_opacity=0.7,
        tooltip=tooltip_text
    ).add_to(high_needs_group)

In [51]:
# Add markers for "Students Without High Needs" (all districts where the student group is not "Students With High Needs")
for index, row in districts_df[districts_df['Student group'] != 'Students With High Needs'].iterrows():
    district_name = row['District name']
    attendance_rate = row['2021-2022 attendance rate - year to date'] * 100  # Convert to percentage
    latitude = row['Latitude']
    longitude = row['Longitude']
    student_count = row['2021-2022 student count - year to date']
    
    # Ensure we have a matching size for the dot
    dot_size = scaled_sizes.iloc[index] if index in scaled_sizes.index else 5  # Default size if mismatch
    
    # Get the color based on attendance rate
    dot_color = get_color(attendance_rate)
    
    tooltip_text = f"District: {district_name}<br>Attendance Rate: {attendance_rate}%"

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=dot_size,  # Adjust size of the dot
        color=dot_color,  # Border color
        fill=True,
        fill_color=dot_color,  # Fill color
        fill_opacity=0.7,
        tooltip=tooltip_text
    ).add_to(without_high_needs_group)

In [52]:
# Add the feature groups to the map
all_students_group.add_to(m)
high_needs_group.add_to(m)
without_high_needs_group.add_to(m)

# Add a layer control to toggle between the groups
folium.LayerControl().add_to(m)

In [53]:
# Save and display the map
m.save('interactive_map.html')
m